In [4]:
!pip3 install docplex
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.9 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 498 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.4 MB 2.8 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import docplex.mp.model as cpx
import pandas as pd

In [6]:
def parse_data(csv_file):
  ''' Returns a clean pandas dataframe containing the stock data in csv_file '''
  df = pd.read_csv(csv_file)
  df = df.dropna(axis=1).set_index('Date', inplace=False)
  df.index = pd.to_datetime(df.index)
  df.sort_index(inplace=True)
  idx = pd.date_range(df.index[0], df.index[-1])
  df = df.reindex(idx, method='bfill')
  return df

In [8]:
df = parse_data('snp.csv')

In [9]:
index = parse_data('index.csv')

In [22]:
index

,Close/Last,Volume,Open,High,Low
2017-11-29,2626.07,--,2627.82,2634.89,2620.32
2017-11-30,2647.58,--,2633.93,2657.74,2633.93
2017-12-01,2642.22,--,2645.10,2650.62,2605.52
2017-12-02,2639.44,--,2657.19,2665.19,2639.03
2017-12-03,2639.44,--,2657.19,2665.19,2639.03
...,...,...,...,...,...
2022-11-24,4026.12,--,4023.34,4034.02,4020.76
2022-11-25,4026.12,--,4023.34,4034.02,4020.76
2022-11-26,3963.94,--,4005.36,4012.27,3955.77
2022-11-27,3963.94,--,4005.36,4012.27,3955.77


In [12]:
df.head()

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
2017-01-03,178.050003,47.500000,39.049999,62.410000,112.360001,116.459999,36.639999,46.189999,103.480003,103.500000,...,123.239998,234.449997,87.459999,40.619999,49.650002,63.209999,86.250000,100.320389,43.180000,53.590000
2017-01-04,178.320007,47.919998,39.360001,63.290001,115.739998,116.739998,37.360001,46.110001,104.139999,103.660004,...,124.760002,236.330002,90.279999,40.799999,50.389999,63.439999,87.029999,101.242722,43.799999,54.110001
2017-01-05,177.710007,47.700001,39.700001,63.770000,114.809998,114.989998,37.939999,45.770000,105.910004,103.040001,...,125.959999,232.259995,91.440002,40.799999,49.930000,63.650002,84.750000,101.893204,43.090000,53.930000
2017-01-06,178.229996,47.720001,40.779999,63.790001,115.419998,116.300003,37.910000,44.720001,108.300003,103.110001,...,126.779999,231.500000,92.430000,40.919998,49.580002,64.419998,85.959999,101.902916,43.369999,54.099998
2017-01-07,177.270004,47.400002,40.740002,64.209999,117.110001,115.000000,37.700001,44.750000,108.570000,102.470001,...,126.010002,230.350006,92.750000,40.299999,49.369999,64.599998,85.970001,103.883492,42.900002,53.950001


In [17]:
returns = df.pct_change(7)[df.index.dayofweek == 0].dropna()
index_returns = index['Close/Last'].pct_change(7)[index.index.dayofweek == 0].dropna()

In [23]:
returns

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
2017-01-16,-0.000056,0.009916,0.003927,-0.036599,-0.026385,0.008957,0.033156,-0.001341,-0.005250,0.005855,...,-0.005396,0.044845,-0.004421,0.022333,-0.023699,-0.006656,-0.034780,0.078972,-0.026340,-0.013531
2017-01-23,0.007052,0.002089,-0.013692,-0.014549,-0.021487,-0.010428,-0.010783,0.000448,0.027500,-0.008441,...,0.002234,0.021398,0.003249,-0.009466,0.006224,0.003740,0.014582,-0.011174,0.012928,0.009959
2017-01-30,-0.017310,0.010006,0.016609,-0.008694,-0.048579,-0.001655,0.028030,-0.012972,0.025683,-0.002250,...,-0.000318,0.035919,0.106973,-0.001960,0.020619,0.019717,0.003445,0.010774,-0.001182,0.009674
2017-02-06,-0.001824,0.023117,0.034626,0.003641,0.040980,-0.003926,0.014643,-0.002493,0.005623,-0.053937,...,-0.015131,-0.004751,-0.053925,0.018905,-0.046869,0.009440,-0.016690,0.016033,0.016801,0.027640
2017-02-13,0.033809,0.007666,0.018619,-0.002308,-0.028235,0.038623,0.137099,-0.007042,0.027870,0.021872,...,0.011886,0.003512,0.020099,0.012771,0.023315,0.027300,0.027086,0.004350,0.025832,-0.011296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-29,-0.019092,-0.023430,0.022114,0.010722,-0.048312,0.006936,-0.030386,-0.024240,0.020666,-0.003885,...,-0.013065,0.005743,-0.120203,-0.010858,-0.029857,-0.009647,0.007015,-0.051605,-0.020226,-0.003521
2021-12-06,0.005504,0.014098,0.034679,0.038156,-0.034376,-0.009814,-0.036810,-0.026230,-0.095201,-0.015685,...,0.009158,0.008818,0.023012,-0.006251,-0.025040,0.028902,-0.037846,0.016113,-0.016022,-0.002595
2021-12-13,-0.014786,0.018780,0.026346,0.042522,0.013760,0.042763,0.010501,0.024243,0.058292,0.012145,...,-0.002409,0.013801,-0.001649,0.034980,0.011599,0.025374,0.035535,-0.000239,-0.042273,0.042113
2021-12-20,-0.011112,-0.051113,-0.007208,0.034385,0.012292,0.021339,0.052811,-0.006497,-0.164864,-0.024083,...,0.000043,-0.034566,-0.046951,-0.005040,-0.065930,-0.004314,-0.056531,-0.052367,-0.028936,0.008177


In [24]:
index_returns

2017-12-11    0.007786
2017-12-18    0.011342
2017-12-25   -0.003591
2018-01-01    0.005712
2018-01-08    0.019252
                ...   
2022-10-31    0.019656
2022-11-07   -0.016834
2022-11-14    0.039521
2022-11-21   -0.001847
2022-11-28    0.003544
Name: Close/Last, Length: 260, dtype: float64

Variables:

$x_0, x_1,\dots, x_n$: weights of stocks in portfolio, sum should be 1

---

Returns defined as: $$r_{i,t} = \frac{P_{i,t} - P_{i,t-1}}{P_{i,t-1}}$$
$P_{i,t}$ is the closing price for stock $i$ at day $t$.

$T=150$: 150 week in-sample size

### Omega Ratio

In [19]:
def expected_return(asset, returns_df):
    return returns_df[asset].mean()

In [20]:
def expected_return_portfolio(portfolio_weights, returns_df):
    output = 0
    for i in range(len(portfolio_weights)):
      output += expected_return(returns_df.columns[i], returns_df) * portfolio_weights[i]
    return output

In [21]:
def omega(L, portfolio_weights):
    ''' Returns the ratio of the expected returns above a certain threshold, L, to the expected returns below a threshold '''
    return 1 + 

SyntaxError: invalid syntax (4008953619.py, line 3)

In [63]:
opt_model = cpx.Model(name="Omega Model")

In [64]:
L = float(index_returns.mean())
T = 150
n = returns.shape[1]
p = 1./150
x_vars  = {n: opt_model.continuous_var(lb=0, ub= 1,
                                 name="x_{0}".format(n)) 
for n in range(n)}
theta = opt_model.continuous_var(lb=0, name="theta")
d_vars = {t: opt_model.continuous_var(name="d_{0}".format(t)) for t in range(T)}
z = opt_model.continuous_var(name="z")
xt_vars = {n: opt_model.continuous_var(name="xt_{0}".format(n)) for n in range(n)}
dt_vars = {t: opt_model.continuous_var(name="dt_{0}".format(t)) for t in range(T)}

In [65]:
constraints = {"d_{0}".format(t) : opt_model.add_constraint(ct=opt_model.sum(opt_model.max(0, float(returns.iloc[t, i]) * x_vars[i]) for i in range(n)) == L - d_vars[t], ctname="d_{0}".format(t)) for t in range(T)}

In [66]:
constraints["z"] = opt_model.add_constraint(ct = opt_model.sum(p*d_vars[t] for t in range(T))*z == 1, ctname="z")

In [67]:
for i in range(n):
    constraints["xt_{0}".format(i)] = opt_model.add_constraint(ct = x_vars[i]*z == xt_vars[i], ctname="xt_{0}".format(i))

In [68]:
for t in range(T):
    constraints["dt_{0}".format(t)] = opt_model.add_constraint(ct = d_vars[t]*z == dt_vars[t], ctname= "dt_{0}".format(t))

In [70]:
constraints[3] = opt_model.add_constraint(ct = opt_model.sum(opt_model.sum(float(returns.iloc[t, i])*xt_vars[i]*p for i in range(n)) for t in range(T)) - z*L >= theta, ctname="3")

In [73]:
for t in range(T):
    constraints["5_{0}".format(t)] = opt_model.add_constraint(ct = L*z - opt_model.sum(float(returns.iloc[t, i])*xt_vars[i] for i in range(n)) <= dt_vars[t], ctname="5_{0}".format(t))

In [74]:
constraints[6] = opt_model.add_constraint(ct = opt_model.sum(p*dt_vars[t] for t in range(T)) == 1, ctname="6")

In [75]:
constraints[7] = opt_model.add_constraint(ct = opt_model.sum(xt_vars[i] for i in range(n)) == z, ctname="7")

In [ ]:
constraints[8] = opt_model.add_constraint(ct = z <= 1/e, ctname="8")

### Robust Omega Ratio

In [ ]:
def cutting_plane(returns):
  ''' Applies the cutting plane algorithm using the given returns as data '''
  pass

In [ ]:
opt_model = cpx.Model(name="Test model")